In [2]:
import numpy as np
import pandas as pd
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

import sys
from PIL import Image
sys.modules['Image'] = Image 

from PIL import Image
print(Image.__file__)

import os
import shutil

Using TensorFlow backend.


/home/serg/anaconda3/envs/py36/lib/python3.6/site-packages/PIL/Image.py


In [3]:
data_dir = 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
validation_dir = os.path.join(data_dir, 'validation')

df_train = pd.read_csv('data/labels.csv')
df_test = pd.read_csv('data/sample_submission.csv')

im_size = 200

x_train = []
y_train = []
x_test = []


In [6]:

print(df_train.head(10))

targets_series = pd.Series(df_train['breed'])
# print(targets_series)
one_hot = pd.get_dummies(targets_series, sparse=True)

one_hot_labels = np.asarray(one_hot)


                                 id               breed
0  000bec180eb18c7604dcecc8fe0dba07         boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97               dingo
2  001cdf01b096e06d78e9e5112d419397            pekinese
3  00214f311d5d2247d5dfe4fe24b2303d            bluetick
4  0021f9ceb3235effd7fcde7f7538ed62    golden_retriever
5  002211c81b498ef88e1b40b9abf84e1d  bedlington_terrier
6  00290d3e1fdd27226ba27a8ce248ce85  bedlington_terrier
7  002a283a315af96eaea0e28e7163b21b              borzoi
8  003df8b8a8b05244b1d920bb6cf451f9             basenji
9  0042188c895a2f14ef64a918ed9c7b64  scottish_deerhound


In [6]:
for index, row in df_train[:10].iterrows():
    filename = row['id'] + '.jpg'
    cl = row['breed']
    
    train_path_orig = os.path.join(train_dir, filename)
    train_path_class = os.path.join(train_dir, cl)
    train_path_new = os.path.join(train_path_class, filename)
    print(train_path_orig, train_path_class, train_path_new)
    
    val_path_orig = os.path.join(validation_dir, filename)
    val_path_class = os.path.join(validation_dir, cl)
    val_path_new = os.path.join(val_path_class, filename)
    print(val_path_orig, val_path_class, val_path_new)

    if os.path.isfile(train_path_orig):
        print('yey')
        if not os.path.exists(train_path_class):
            os.makedirs(train_path_class)
        shutil.move(train_path_orig, train_path_new)
    
    if os.path.isfile(val_path_orig):
        print('yey')
        if not os.path.exists(val_path_class):
            os.makedirs(val_path_class)
        shutil.move(val_path_orig, val_path_new)


data/train/000bec180eb18c7604dcecc8fe0dba07.jpg data/train/boston_bull data/train/boston_bull/000bec180eb18c7604dcecc8fe0dba07.jpg
data/validation/000bec180eb18c7604dcecc8fe0dba07.jpg data/validation/boston_bull data/validation/boston_bull/000bec180eb18c7604dcecc8fe0dba07.jpg
data/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg data/train/dingo data/train/dingo/001513dfcb2ffafc82cccf4d8bbaba97.jpg
data/validation/001513dfcb2ffafc82cccf4d8bbaba97.jpg data/validation/dingo data/validation/dingo/001513dfcb2ffafc82cccf4d8bbaba97.jpg
data/train/001cdf01b096e06d78e9e5112d419397.jpg data/train/pekinese data/train/pekinese/001cdf01b096e06d78e9e5112d419397.jpg
data/validation/001cdf01b096e06d78e9e5112d419397.jpg data/validation/pekinese data/validation/pekinese/001cdf01b096e06d78e9e5112d419397.jpg
data/train/00214f311d5d2247d5dfe4fe24b2303d.jpg data/train/bluetick data/train/bluetick/00214f311d5d2247d5dfe4fe24b2303d.jpg
data/validation/00214f311d5d2247d5dfe4fe24b2303d.jpg data/validation/bluetick da

In [17]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('data/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1
    
for f in tqdm(df_test['id'].values):
    img = cv2.imread('data/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))
    
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

num_class = y_train_raw.shape[1]

print(1)

  0%|          | 0/10222 [00:00<?, ?it/s]

error: OpenCV(3.4.4) /io/opencv/modules/imgproc/src/resize.cpp:3784: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [18]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)

NameError: name 'x_train_raw' is not defined

In [7]:
# Create the base pre-trained model
# Can't download weights in the kernel
# base_model = VGG16(
#     weights='imagenet',
#     # weights = None,
#     include_top=False, input_shape=(im_size, im_size, 3))

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(im_size, im_size, 3), pooling=None)
for layer in base_model.layers[:-4]:
    layer.trainable = True
# Create the model

model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(base_model)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

# # Add a new top layer
# x = base_model.output
# x = Flatten()(x)
# predictions = Dense(num_class, activation='softmax')(x)
# 
# # This is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)
# 
# # First: train only the top layers (which were randomly initialized)
# for layer in base_model.layers:
#     layer.trainable = False
# 
# model.compile(loss='categorical_crossentropy', 
#               # optimizer='adam', 
#               optimizer=optimizers.RMSprop(lr=1e-4), 
#               metrics=['accuracy'])

# callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
# model.summary()


/home/serg/anaconda3/envs/py36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              102761472 
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               123000    
Total params: 126,472,184
Trainable params: 126,419,064
Non-trainable params: 53,120
_________________________________________________________________


In [8]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(im_size, im_size),
        batch_size=train_batchsize,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(im_size, im_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


Found 7422 images belonging to 120 classes.
Found 2800 images belonging to 120 classes.


In [9]:
# model.fit(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid), verbose=1)
# Train the model


model.compile(loss='categorical_crossentropy', 
              # optimizer='adam', 
              optimizer=optimizers.RMSprop(lr=1e-4), 
              metrics=['accuracy'])


history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/30


 1/74 [..............................] - ETA: 1:17:41 - loss: 8.4611 - acc: 0.0000e+00

 2/74 [..............................] - ETA: 1:06:03 - loss: 9.6683 - acc: 0.0200    

 3/74 [>.............................] - ETA: 56:39 - loss: 10.0971 - acc: 0.0200 

 4/74 [>.............................] - ETA: 50:51 - loss: 9.8145 - acc: 0.0200 

...

 2/74 [..............................] - ETA: 52:53 - loss: 0.4359 - acc: 0.8700

 3/74 [>.............................] - ETA: 45:43 - loss: 0.4904 - acc: 0.8567

 4/74 [>.............................] - ETA: 40:36 - loss: 0.4619 - acc: 0.8500

 5/74 [=>............................] - ETA: 37:28 - loss: 0.4867 - acc: 0.8520

 6/74 [=>............................] - ETA: 35:32 - loss: 0.4911 - acc: 0.8517

In [1]:
x_test = []
for f in tqdm(df_test['id'].values):
    img = cv2.imread('data/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))
    
x_test  = np.array(x_test, np.float32) / 255.

NameError: name 'tqdm' is not defined

In [ ]:
preds = model.predict(x_test, verbose=1)

   32/10357 [..............................] - ETA: 2:51

   64/10357 [..............................] - ETA: 2:35

   96/10357 [..............................] - ETA: 2:24

  128/10357 [..............................] - ETA: 2:19

  160/10357 [..............................] - ETA: 2:16

...

10240/10357 [============================>.] - ETA: 1s

10272/10357 [============================>.] - ETA: 1s

10304/10357 [============================>.] - ETA: 0s

10336/10357 [============================>.] - ETA: 0s

10357/10357 [==============================] - 144s 14ms/step


In [73]:
sub = pd.DataFrame(preds)

# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
print('started')
sub.insert(0, 'id', df_test['id'])
sub.to_csv('data/submission_resnet.csv', encoding='utf-8', index=False)
print('success')

started


success
